In [1]:
from character_mapping import Character_Map 
import theano 
import theano.tensor as T
import numpy as np 

text_test = './../texts/melville.txt'
char_map_obj = Character_Map(text_test,'mapping.dat',overwrite=True, break_line=None)
char_map_obj.k_map()
x_, y_, shared_x, shared_y = char_map_obj.gen_x_and_y(filename=None)

# print(shared_y.get_value().shape[1])
nh = 100
nx = len(char_map_obj.unique_char)
ny = nx 

Time creating character mapping and pickling: 1.9811 sec
Time creating k map 3.767 sec
Time creating arrays: 2.024 sec


<TensorType(float64, 3D)>
<TensorType(float64, 3D)>


In [ ]:
import theano 
import theano.tensor as T 
import numpy as np 
from character_mapping import Character_Map
import time
try:
    reload
except NameError:
    from imp import reload
    
import RNN  
from RNN import RNNClass

reload(RNN)
from RNN import RNNClass

trainer = RNNClass(nh, nx, ny)
# trainer.train_no_index(training_data=(shared_x,shared_y),
#                        learning_rate=0.01,
#                        n_epochs=100,mini_batch_size=100)

trainer.train_index(training_data=(shared_x,shared_y),
                       learning_rate=0.001,
                       n_epochs=100,mini_batch_size=1000)

# x = T.matrix('x')
# y = T.matrix('y')


# cost = trainer.cross_entropy_loss(x,y)

# cost_fn = theano.function(
#     inputs = [x,y],
#     outputs = cost
#     )
# print("cost compiled")


Saving initial parameters
Function compiled


30 out of 79 minibatches done, took 3.314
30 out of 79 minibatches done, took 3.392
30 out of 79 minibatches done, took 3.461
Epoch number 0, took 280.850 sec
Pickling epoch number 0 took 0.033 sec
30 out of 79 minibatches done, took 3.820

In [71]:
import numpy as np
import time 
xs = T.tensor3('xs')
ys = T.tensor3('ys')
x0 = T.matrix('x0')

results1, updates1 = theano.scan(lambda prev_cost, xi, yi: trainer.cross_entropy_loss(xi, yi) + prev_cost,
                              sequences = [xs,ys],
                              outputs_info = trainer.cross_entropy_loss(xs[0],ys[0]))
results2, updates2 = theano.scan(lambda xi, yi: trainer.cross_entropy_loss(xi, yi),
                              sequences = [xs,ys])
                              #outputs_info = trainer.cross_entropy_loss(xs[0],ys[0]))
loss_fn1 = theano.function(
    inputs = [xs,ys],
    outputs = -T.mean(results1[-1])
)
loss_fn2 = theano.function(
    inputs = [xs,ys],
    outputs = -T.mean(results2)
)
print("loss_fn compiled")
t1 = time.time()
loss1 = loss_fn1(shared_x.get_value()[0:500], shared_y.get_value()[0:500])
print(time.time()-t1)
t2 = time.time()
loss2 = loss_fn2(shared_x.get_value()[0:500], shared_y.get_value()[0:500])
print(time.time()-t2)
# loss2 = -np.mean(loss1, axis=0)
# loss2.shape

loss_fn compiled
1.41085410118
1.41524791718
